# Huấn luyện mô hình NER với spaCy 3.x

## I. Import

In [1]:
import spacy
from spacy.tokens import DocBin
from spacy.training.example import Example
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import json
from tqdm import tqdm
from spacy.util import minibatch

In [2]:
# Chuyển sang GPU nếu có
spacy.require_gpu()

if spacy.prefer_gpu():
    print("Using GPU")
else:
    print("Not using GPU, falling back to CPU")

from thinc.api import get_current_ops
print("Current device:", get_current_ops().__class__.__name__)

Using GPU
Current device: CupyOps


## II. Load Data

In [3]:
# Load dữ liệu
with open("./data/ner_data.json", "r", encoding="utf-8") as f:
    raw = json.load(f)["dataset"]

In [4]:
# Tạo mô hình trắng
nlp = spacy.blank("vi")
ner = nlp.add_pipe("ner")

In [5]:
# Bổ sung nhãn từ dữ liệu
for item in raw:
    for _, _, label in item["entities"]:
        ner.add_label(label)

In [6]:
# Chuẩn bị dữ liệu huấn luyện, loại bỏ entity sai alignment
TRAIN_DATA_CLEAN = []

for item in raw:
    text = item["text"]
    entities = item["entities"]
    doc = nlp.make_doc(text)
    try:
        tags = offsets_to_biluo_tags(doc, entities)
        if "-" not in tags:  # hợp lệ
            example = Example.from_dict(doc, {"entities": entities})
            TRAIN_DATA_CLEAN.append(example)
    except:
        continue  # bỏ qua nếu sai lớn hơn

print(f"Số mẫu hợp lệ: {len(TRAIN_DATA_CLEAN)}")

D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hệ Du lịch (Chuyên ngành Hướng dẫn du lịch) đào tạ..." with entities "[[3, 43, 'Major'], [58, 86, 'Programme']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Hệ Dược đào tạo ngành Tiên tiến: Luật (Chuyên ngàn..." with entities "[[3, 7, 'Major'], [22, 65, 'Programme']]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
D:\Programs\Python\Python311\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be alig

Số mẫu hợp lệ: 20623


## III. Training

In [7]:
# Huấn luyện mô hình
optimizer = nlp.begin_training()
for i in range(10):
    random.shuffle(TRAIN_DATA_CLEAN)
    losses = {}
    print(f"Epoch {i+1}")
    batches = minibatch(TRAIN_DATA_CLEAN, size=32)  # batch size
    for batch in tqdm(batches, desc=f"Training Epoch {i+1}"):
        nlp.update(batch, drop=0.3, sgd=optimizer, losses=losses)
    print(f"Loss: {losses}")

Epoch 1


Training Epoch 1: 645it [00:36, 17.62it/s]


Loss: {'ner': 9080.839780246164}
Epoch 2


Training Epoch 2: 645it [00:48, 13.18it/s]


Loss: {'ner': 55.36435844555748}
Epoch 3


...

Loss: {'ner': 23.344967616084833}
Epoch 9


Training Epoch 9: 645it [03:17,  3.26it/s]


Loss: {'ner': 35.63873693676629}
Epoch 10


Training Epoch 10: 645it [03:15,  3.30it/s]

Loss: {'ner': 28.20683312001551}


## IV. Save model

In [8]:
# nlp.to_disk("../ner_model")
# print("Mô hình đã lưu vào thư mục ner_model")

# Lưu mô hình
nlp.to_disk("../ner_model_cleaned")
print("Đã lưu mô hình vào thư mục ner_model_cleaned")

Đã lưu mô hình vào thư mục ner_model_cleaned


## V. Save invalid NER data

In [9]:
# with open("data/invalid_ner_data.json", "w", encoding="utf-8") as f:
#     json.dump(invalid_data, f, ensure_ascii=False, indent=2)
# print(f"Đã lưu {len(invalid_data)} câu lỗi vào file: data/invalid_ner_data.json")

## VI. Test

In [10]:
ner_nlp = spacy.load("../ner_model_cleaned")

In [11]:
# ======== METHODS =========
def extract_entities(text: str):
    doc = ner_nlp(text)
    return [
        {
            "text": ent.text,
            "start": ent.start_char,
            "end": ent.end_char,
            "label": ent.label_
        }
        for ent in doc.ents
    ]

In [22]:
text = "ngành khmt thuộc hệ đào tạo nào" 
entities = extract_entities(text)

print(entities)

[{'text': 'khmt', 'start': 6, 'end': 10, 'label': 'Major'}]
